In [6]:
using DynamicPolynomials, LinearAlgebra, SparseArrays

println("***Problem setting***")

n=6

println("Number of variable: n=",n)
println("====================")

@polyvar x[1:n]# variables

function generate_random_poly(T::UnitRange{Int64})
    v=reverse(monomials(x[T],0:2))
    c=2*rand(Float64,length(v)).-1
    return c'*v
end

# unit sphere constraint
u=3
p=floor(Int64,n/u)

I=Vector{UnitRange{Int64}}(undef,p)
I[1]=1:u
I[2:p-1]=[u*(j-1):u*j for j in 2:p-1]
I[p]=u*(p-1):n

# random quadratic objective function f
f=sum(generate_random_poly(I[j]) for j in 1:p)


g=-(x.^2).+1/u



m=length(g)
println("Number of inequality constraints: m=",m)
println("====================")

l=ceil(Int64, n/7)

r=floor(Int64,l/p)
W=[(j-1)*r+1:j*r for j in 1:p-1]
append!(W,[(p-1)*r+1:l])

h=Vector{Polynomial{true,Float64}}(undef,l)

#=randx=[2*rand(length(I[j])).-1 for j in 1:p]# create a feasible solution
randx=[randx[j]/norm(randx[j]) for j in 1:p]=#

randx=2*rand(n).-1
randx=randx./sqrt(u)

for j in 1:p
    for i in W[j]
        h[i]=generate_random_poly(I[j])
        h[i]-=h[i](x => randx) #make constraints feasible
    end
end

l=length(h)
println("Number of equality constraints: l=",l)
println("====================")

k=2

println("Relaxed order: k=",k)
println("====================")

***Problem setting***
Number of variable: n=6
Number of inequality constraints: m=6
Number of equality constraints: l=1
Relaxed order: k=2


In [7]:
include("../src/ctpPOP.jl")
using .ctpPOP

n,m,l,lmon_g,supp_g,coe_g,lmon_h,supp_h,coe_h,lmon_f,supp_f,coe_f,dg,dh=ctpPOP.get_info(x,f,g,h,sparse=true);

In [8]:
include("../src/ctpPOP.jl")
using .ctpPOP

opt_val1=ctpPOP.POP_CS_CGAL(n,m,l,lmon_g,supp_g,coe_g,lmon_h,supp_h,coe_h,lmon_f,supp_f,coe_f,dg,dh,k,
    maxit=Int64(1e10),tol=1e-2,use_eqcons_to_get_constant_trace=false,check_tol_each_iter=true)

------------------------------------------------------
The clique sizes of varibles:
[4, 3]
[1, 1]
------------------------------------------------------
  Number of cliques: p=2
  Largest clique size: u=4


  Largest constant trace: a=4.111111111111111
  Number of blocks: omega=9
  Number of equality consraints: zeta=181
  Size of the largest block: s^max=15
Modeling time:
  0.300188 seconds (214.51 k allocations: 12.019 MiB)
iter=1   val=-5.85353220044705   gap=0.029777333624250224   feas=4.9394572334012
iter=2   val=-4.549808976548115   gap=0.10711285550746319   feas=3.2881765028410683
iter=4   val=-2.894420660913574   gap=0.3091601393458129   feas=2.398968417295035
iter=8   val=-2.973222975562018   gap=0.05017757490768229   feas=0.9904213647827789
iter=16   val=-2.364597896619191   gap=0.16920558125109555   feas=0.5135656751242373
iter=32   val=-2.4609703127139713   gap=0.010264017364901843   feas=0.17757174124809913
iter=64   val=-2.378887316601847   gap=0.037898352986812024   feas=0.05260189374605285
iter=128   val=-2.401153599741716   gap=0.023887157202859874   feas=0.023867568156076716
iter=221   val=-2.4167574661622897   gap=0.006942137502166342   feas=0.009963100892794832
tol sat

-2.4167574661622897

In [9]:
using TSSOS


@time begin
    opt,sol,data=cs_tssos_first(Vector{SparseMatrixCSC{UInt8,UInt32}}([[supp_f];supp_g;supp_h]),[[coe_f];coe_g;coe_h],n,k,[dg;dh],numeq=l,CS="MD",TS="block",CTP=false);
    #opt,sol,data=cs_tssos_higher!(data,TS="block");
end

------------------------------------------------------
The clique sizes of varibles:
[4, 3]
[1, 1]
------------------------------------------------------
Problem
  Name                   :                 
  Objective sense        : max             
  Type                   : CONIC (conic optimization problem)
  Constraints            : 100             
  Cones                  : 0               
  Scalar variables       : 16              
  Matrix variables       : 10              
  Integer variables      : 0               

Optimizer started.
Presolve started.
Linear dependency checker started.
Linear dependency checker terminated.
Eliminator started.
Freed constraints in eliminator : 0
Eliminator terminated.
Eliminator - tries                  : 1                 time                   : 0.00            
Lin. dep.  - tries                  : 1                 time                   : 0.00            
Lin. dep.  - number                 : 0               
Presolve terminated. Time: 

(-2.4172637669149273, nothing, TSSOS.mdata_type(6, 0, 7, [2, 2, 2, 2, 2, 2, 2], SparseMatrixCSC{UInt8,UInt32}[
  [1,  1]  =  0x02
  [1,  2]  =  0x01
  [2,  2]  =  0x01
  [1,  3]  =  0x01
  [3,  3]  =  0x01
  [2,  4]  =  0x02
  [2,  5]  =  0x01
  [3,  5]  =  0x01
  [3,  6]  =  0x02
  [3,  7]  =  0x01
  [4,  7]  =  0x01
  [3,  8]  =  0x01
  ⋮
  [5, 11]  =  0x01
  [4, 12]  =  0x01
  [6, 12]  =  0x01
  [5, 13]  =  0x02
  [5, 14]  =  0x01
  [6, 14]  =  0x01
  [6, 15]  =  0x02
  [1, 16]  =  0x01
  [2, 17]  =  0x01
  [3, 18]  =  0x01
  [4, 19]  =  0x01
  [5, 20]  =  0x01
  [6, 21]  =  0x01, 
  [1, 1]  =  0x02, 
  [2, 1]  =  0x02, 
  [3, 1]  =  0x02, 
  [4, 1]  =  0x02, 
  [5, 1]  =  0x02, 
  [6, 1]  =  0x02, 
  [3,  1]  =  0x02
  [3,  2]  =  0x01
  [4,  2]  =  0x01
  [3,  3]  =  0x01
  [5,  3]  =  0x01
  [3,  4]  =  0x01
  [6,  4]  =  0x01
  [4,  5]  =  0x02
  [4,  6]  =  0x01
  [5,  6]  =  0x01
  [4,  7]  =  0x01
  [6,  7]  =  0x01
  [5,  8]  =  0x02
  [5,  9]  =  0x01
  [6,  9]  =  0x01
  [

In [10]:
include("../src/ctpPOP.jl")
using .ctpPOP

optval=ctpPOP.POP_NLP(n,m,l,lmon_g,supp_g,coe_g,lmon_h,supp_h,coe_h,lmon_f,supp_f,coe_f)

This is Ipopt version 3.13.2, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:       20
Number of nonzeros in inequality constraint Jacobian.:        6
Number of nonzeros in Lagrangian Hessian.............:       31

Total number of variables............................:        6
                     variables with only lower bounds:        0
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equality constraints.................:        1
Total number of inequality constraints...............:        6
        inequality constraints with only lower bounds:        6
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:        0

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0 -

-2.4172637894993945